In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as Dateformatter

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt
from pandas.plotting import table
import sys

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

##### Design a query to retrieve the last 12 months of precipitation data and plot the results


In [ ]:
# Identify the most recent date in the dataset and format as a date
results_A = session.query(Measurement.date, Measurement.prcp).all()
df_A = pd.DataFrame(results_A, columns=['date', 'precipitation'])
df_A1 = df_A.sort_values(by=['date'], ascending=False)
recent_dt = df_A1.iloc[0]['date']
recent_pt = []
recent_pt = recent_dt.split("-")
recent_dt1 = dt.date(int(recent_pt[0]), int(recent_pt[1]), int(recent_pt[2]))

In [ ]:
# Calculate the date 1 year ago from the last data point in the database
year_back = recent_dt1 - dt.timedelta(days=365)

In [ ]:
# Perform a query to retrieve the data and precipitation scores
year_back_dat = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= year_back).all()

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
year_back_dat_df = pd.DataFrame(year_back_dat, columns=['date', 'precipitation'])
year_back_dat_df.set_index(year_back_dat_df['date'], inplace=True)

In [ ]:
# Sort the dataframe by date
year_back_dat_df = year_back_dat_df.sort_values(by=['date'], ascending=True)

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
plt.figure(figsize=(20,10))
plt.plot_date(x=year_back_dat_df['date'], y=year_back_dat_df['precipitation'])
plt.title("Precipitation Over Most Recent 365 Days")
plt.xlabel("Date of Precipitation Readings")
plt.ylabel("Inches of Daily Rainfall")

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data

year_back_dat_df.describe()

In [ ]:
# Design a query to show how many stations are available in this dataset?
session.query(func.count(Station.station)).all()

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
station_act = session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
most_act = station_act[0][0]
station_act

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
    filter(Measurement.station == most_act).all()

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
results = session.query(Measurement.tobs).\
    filter(Measurement.station == most_act).\
    filter(Measurement.date >= year_back).all()
temp_obs = pd.DataFrame(results, columns=['tobs'])
plt.figure(figsize=(10,7))
hist, bins = np.histogram(temp_obs, bins=50)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.xlabel('Observed Temperature')
plt.ylabel('Number of Observations in Past Year')
plt.show()

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 

# This code assumes that trips will only be taken after 5/19/2019 (today), otherwise there would be no need to estimate temps. 
#   No estimation data would be included if we only looked 365 days back, so at a minimum we must look back two years
#   If the trip was being planned for times beyond August 23, 2019 we must look back three years.   

desired_dt = input("Please enter the date you would like to begin your trip \
                  \nusing the format YYYY-MM-DD then press Enter:  ")

In [ ]:
trip_start = dt.datetime.strptime(desired_dt, '%Y-%m-%d')

In [ ]:
if  trip_start < dt.datetime.today() :
    print(f"We apologize, the date you have planned for your trip ({trip_start}) occurs in the past")
    print(f"and at this time, we cannot accommodate your request.  Please check back later")
    print(f"Mr. Peabody should be done with repairs to the WayBack Machine any day now.")
    desired_dt = input("Please enter a future date when you would like to begin your trip \
                  \nusing the format YYYY-MM-DD then press Enter:  ")
    trip_start = dt.datetime.strptime(desired_dt, '%Y-%m-%d')
    if trip_start < dt.datetime.today():
        trip_start = dt.datetime.today() + dt.timedelta(days=7)
        print(f"So next week it is: {trip_start}")

In [ ]:
trip_days = input("Please enter the length of your trip in number of days \
                  \ntype in a number from 7 to 100 then press Enter:  ")

In [ ]:
trip_days = int(trip_days)

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 

# This code assumes that trips will only be taken after 5/19/2019 (today), otherwise there would be no need to estimate temps. 
#   No estimation data would be included if we only looked 365 days back, so at a minimum we must look back two years
#   If the trip was being planned for times beyond August 23, 2019 we must look back three years.   

trip_end = trip_start + dt.timedelta(days=trip_days)
recent_dt1 = dt.datetime.strptime(recent_dt, '%Y-%m-%d')

if trip_end > recent_dt1 + dt.timedelta(days=730):
    data_start = trip_start - dt.timedelta(days=1095)
    data_end = trip_end - dt.timedelta(days=1095)
else:        
    data_start = trip_start - dt.timedelta(days=730)
    data_end = trip_end - dt.timedelta(days=730)

tmin, tavg, tmax = calc_temps(data_start.strftime("%Y-%m-%d"), data_end.strftime("%Y-%m-%d"))[0]
print(tmin, tavg, tmax, data_start, data_end)
print(f"\nTRIP AND ESTIMATED TEMPERATURE INFORMATION\n\nTrip Start Date: {trip_start} \nTrip End Date:  {trip_end} \nTrip Length: {trip_days} days\
        \nMinimum Temperature (est): {tmin:3.0f}\nMaximum Temperature (est): {tmax:3.0f}\nAverage Temperature (est): {tavg:3.1f}\
        \nDates Used for Estimates: {data_start} to {data_end}")

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
fig, ax = plt.subplots(figsize=plt.figaspect(1.))
xpos = 1
yerr = tmax-tmin

bar = ax.bar(xpos, tavg, yerr=yerr, alpha=0.5, color='navy', align="center")
ax.set(xticks=range(xpos), xticklabels="c", title="Trip Avg Temp", ylabel="Temp (F)")
ax.margins(.2, .2)
fig.tight_layout()
fig.show()

ax.set()

In [ ]:
# A box and whisker plot seemed more informative and intuitive than the plot described above, so I took a stab at that. 

resultsa = session.query(Measurement.tobs).\
    filter(Measurement.date >= data_start).\
    filter(Measurement.date <= data_end).all()
trip_temp_obs = pd.DataFrame(resultsa, columns=['tobs'])
trip_temp_sd = np.std(trip_temp_obs.values, axis=0, ddof=0)
plt.figure(figsize=(5,7))
plt.boxplot(trip_temp_obs['tobs'])
plt.title("Estimated Temperatures\nDuring Your Trip")
plt.ylabel("Estimated Average Temperature\n(Most Likely Average Temperature\nShown by Boxed Area)")
plt.show()

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Since trip dates cannot be in the past and previous year's dates are not available for any dates from May 19, 2019 (Today),
# I've made the same adjustment described above
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation

fields_sel = [Station.station, Station.name, Station.latitude, 
       Station.longitude, Station.elevation, func.sum(Measurement.prcp)]

results = session.query(*fields_sel).\
    filter(Measurement.station == Station.station).\
    filter(Measurement.date >= data_start).\
    filter(Measurement.date <= data_end).\
    group_by(Station.name).order_by(func.sum(Measurement.prcp).desc()).all()
print(results)




## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    fields_sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*fields_sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

trip_dates = pd.date_range(trip_start, trip_end, freq='D')

# Stip off the year and save a list of %m-%d strings
trip_month_day = trip_dates.strftime('%m-%d')

# Loop through the list of %m-%d strings and calculate the normals for each date
normals = []
for date in trip_month_day:
    normals.append(*daily_normals(date))
    
normals

In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index
normals_df = pd.DataFrame(normals, columns=['tmin', 'tavg', 'tmax'])
normals_df['date'] = trip_dates
normals_df.set_index(['date'],inplace=True)
normals_df.head()


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
plt.figure(figsize=(5,7))
normals_df.plot(kind='area', stacked=False, x_compat=True, alpha=.2)
#plt.tight_layout()
plt.show()